In [1]:
from bs4 import BeautifulSoup
from requests import get

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import math 
import os

pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

pd.options.display.max_colwidth = 100

In [2]:
# Function for remove comma within numbers
def removeCommas(string): 
    string = string.replace(',','')
    return string 

# Scrap data from worldmeter

In [3]:
# Test if we can scrap info from worldometers
# The communication with website is ok if the response is 200
headers = ({'User-Agent':'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})
worldometers = "https://www.worldometers.info/coronavirus/#countries"
response = get(worldometers, headers=headers)
response

<Response [200]>

In [4]:
# Scrap all content from the website
html_soup = BeautifulSoup(response.text, 'html.parser')
# After inspect the website content, data are stored inside tag 'tbody' and table header is 'thead'
table_contents = html_soup.find_all('tbody')
table_header = html_soup.find_all('thead')

# Header for the table
header = []
for head_title in table_header[0].find_all('th'):    
    header.append(str(head_title.contents))

# Save value into columns
CountryName = []
TotalCases = []
NewCases = []
TotalDeaths = []
NewDeaths = []
TotalRecovered = []
ActiveCases = []
SeriousCritical = []

for row in table_contents[0].find_all('tr'):
    cells = row.find_all('td')
    if len(cells[0].find_all('a')) >= 1:
        CountryName.append(cells[0].find_all('a')[0].contents[0])
    elif len(cells[0].find_all('span')) >= 1:
        CountryName.append(cells[0].find_all('span')[0].contents[0])   
    else:
        CountryName.append(cells[0].contents[0])
    
    
    if len(cells[1].contents) >=1:
        TotalCases.append(cells[1].contents[0])
    else:
        TotalCases.append(0)
    
    if len(cells[2].contents) >= 1:
        NewCases.append(cells[2].contents[0])
    else:
        NewCases.append(0)
        
    
    if len(cells[3].contents) >= 1:
        TotalDeaths.append(cells[3].contents[0])
    else:
        TotalDeaths.append(0)

    
    if len(cells[4].contents) >= 1:
        NewDeaths.append(cells[4].contents[0])
    else:
        NewDeaths.append(0)
    
    if len(cells[5].contents) >= 1:
        TotalRecovered.append(cells[5].contents[0])
    else:
        TotalRecovered.append(0)
        
    if len(cells[6].contents) >= 1:
        ActiveCases.append(cells[6].contents[0])
    else:
        ActiveCases.append(0)
    
    if len(cells[7].contents) >= 1:
        SeriousCritical.append(cells[7].contents[0])
    else:
        SeriousCritical.append(0)
        
        
CaseTable = pd.DataFrame({header[0]: CountryName,
                          header[1]: TotalCases,
                          header[2]: NewCases,
                          header[3]: TotalDeaths,
                          header[4]: NewDeaths,                          
                          header[5]: TotalRecovered,
                          header[6]: ActiveCases,
                          header[7]: SeriousCritical,
                          })  

# CaseTable.head(40)

In [5]:
caseTableSimple = CaseTable[[CaseTable.columns[0], CaseTable.columns[1], CaseTable.columns[3], CaseTable.columns[5]]]
caseTableSimple.columns = ['Country/Region', 'Confirmed', 'Deaths', 'Recovered']
# Set data type as string first for manuipulation
caseTableSimple = caseTableSimple.astype({'Country/Region':str,'Confirmed':str,'Deaths':str, 'Recovered':str})
# Remove the last row of total number (changed on 20200310, worldmeter moved this row as next tbody)
#caseTableSimple = caseTableSimple.iloc[:-1,:]
# Remove lead and tail space for each element
caseTableSimple = caseTableSimple.apply(lambda x: x.str.strip())
# Remove comma for each element
caseTableSimple = caseTableSimple.applymap(removeCommas)
# Replace empty str with zero. This include row of 'Diamond Princess' (its name is empty)
caseTableSimple = caseTableSimple.replace('', '0')
# After string manipulation, convert data type as correct type
caseTableSimple = caseTableSimple.astype({'Country/Region':'str',
                                          'Confirmed':'int',
                                          'Deaths':'int',
                                          'Recovered':'int',                                          
                                         })
# Data for these countries come from other source
removeRegion = []
for i in removeRegion:
    caseTableSimple.drop(caseTableSimple[caseTableSimple['Country/Region'] == i].index, axis=0, inplace=True)

# Change Country name the same as my old data 
if 'S. Korea' in list(caseTableSimple['Country/Region']):
    caseTableSimple = caseTableSimple.replace('S. Korea', 'South Korea')

# Add column 'Province/State' with empty value
caseTableSimple['Province/State'] =''

# In my old data, 'Diamond Princess' is represented by 'Yokohama' in the column of 'Province/State'
if 'Diamond Princess' in list(caseTableSimple['Country/Region']):
    caseTableSimple.at[caseTableSimple.loc[caseTableSimple['Country/Region'] == 'Diamond Princess',].index, 'Province/State'] = 'Yokohama'
    caseTableSimple['Country/Region'].replace({'Diamond Princess':'Japan'}, inplace=True)

# In my old data, 'Belgium' has 'Brussels' in the column of 'Province/State'
if 'Belgium' in list(caseTableSimple['Country/Region']):
    caseTableSimple.at[caseTableSimple.loc[caseTableSimple['Country/Region'] == 'Belgium',].index, 'Province/State'] = 'Brussels'

# In my old data, I used 'Macau' not 'Macao'
if 'Macao' in list(caseTableSimple['Country/Region']):
    caseTableSimple.at[caseTableSimple.loc[caseTableSimple['Country/Region'] == 'Macao',].index, 'Province/State'] = 'Macau'
    caseTableSimple['Country/Region'].replace({'Macao':'Macau'}, inplace=True)

# In my old data, 'Hong Kong' has 'Hong Kong' in the column of 'Province/State'
if 'Hong Kong' in list(caseTableSimple['Country/Region']):
    caseTableSimple.at[caseTableSimple.loc[caseTableSimple['Country/Region'] == 'Hong Kong',].index, 'Province/State'] = 'Hong Kong'

# In my old data, 'Taiwan' has 'Taiwan' in the column of 'Province/State'
if 'Taiwan' in list(caseTableSimple['Country/Region']):
    caseTableSimple.at[caseTableSimple.loc[caseTableSimple['Country/Region'] == 'Taiwan',].index, 'Province/State'] = 'Taiwan'

# In my old data, I used 'United Arab Emirates' not 'UAE'
if 'UAE' in list(caseTableSimple['Country/Region']):
    caseTableSimple['Country/Region'].replace({'UAE':'United Arab Emirates'}, inplace=True)

if 'Réunion' in list(caseTableSimple['Country/Region']):
    caseTableSimple['Country/Region'].replace({'Réunion':'Reunion'}, inplace=True)
    
if 'Curaçao' in list(caseTableSimple['Country/Region']):
    caseTableSimple['Country/Region'].replace({'Curaçao':'Curacao'}, inplace=True)

# In my old data I used US time as Last Update time
currentTime = datetime.now()
lastUpdateTime = currentTime.strftime('%m/%d/%Y %H:%M')
# Remove the first number (This only works for month number less than 10)
lastUpdateTime[1:]
caseTableSimple['Last Update'] = lastUpdateTime[1:]

# Reorder list as all old data
columnList = caseTableSimple.columns.tolist()
columnList =[columnList[i] for i in [4, 0, 5, 1, 2, 3]]
caseTableSimple = caseTableSimple[columnList]

In [6]:
finalTable = caseTableSimple.copy()
# finalTable



finalTable['Confirmed'] = finalTable['Confirmed'].fillna(0).astype(int)
finalTable['Deaths'] = finalTable['Deaths'].fillna(0).astype(int)
finalTable['Recovered'] = finalTable['Recovered'].fillna(0).astype(int)

mask = ((finalTable['Country/Region'].notnull()) | (finalTable['Province/State'].notnull()) | (finalTable['Last Update'].notnull()) |
        (finalTable['Recovered'].notnull()) | (finalTable['Deaths'].notnull()) | (finalTable['Confirmed']))
finalTable = finalTable[mask]

finalTable['Country/Region'] = finalTable['Country/Region'].replace('USA','US')

In [7]:
timeStampe = currentTime.strftime('%m_%d_%Y_%H_%M')
finalTable.to_csv('../data/web_data/{}_webData.csv'.format(timeStampe), index=False)
finalTable.to_csv('../data/web_data/last_capure.csv', index=False)

date_time = currentTime.strftime('%Y-%m-%d-%H-%M')


# Consolidate Data

In [8]:
new_data = pd.read_csv('../data/web_data/last_capure.csv')
new_data = new_data.groupby(by=['Country/Region','Last Update'], as_index=False).sum()

date_day = timeStampe = currentTime.strftime('%Y-%m-%d')

new_data['date'] = date_day

new_data = new_data.rename(columns={'Country/Region':'countryname','Last Update':'Date_last_updated_AEDT'})


cols = ['Date_last_updated_AEDT', 'date', 'countryname', 'Confirmed', 'Deaths', 'Recovered']
new_data = new_data[cols]

In [9]:
new_data.sort_values(by='Confirmed', ascending=False).reset_index(drop=True).head(18)

,Date_last_updated_AEDT,date,countryname,Confirmed,Deaths,Recovered
0,3/25/2020 17:32,2020-03-25,China,81218,3281,73650
1,3/25/2020 17:32,2020-03-25,Italy,74386,7503,9362
2,3/25/2020 17:32,2020-03-25,US,64765,909,393
3,3/25/2020 17:32,2020-03-25,Spain,47611,3445,5367
4,3/25/2020 17:32,2020-03-25,Germany,37323,206,3547
5,3/25/2020 17:32,2020-03-25,Iran,27017,2077,9625
6,3/25/2020 17:32,2020-03-25,France,25233,1331,3900
7,3/25/2020 17:32,2020-03-25,Switzerland,10897,153,131
8,3/25/2020 17:32,2020-03-25,South Korea,9137,126,3730
9,3/25/2020 17:32,2020-03-25,UK,8264,435,135


In [10]:
last_data = pd.read_csv('../data/cumulative_data/covid_last.csv')
mask = last_data['date']!=date_day
last_data = last_data[mask]

In [11]:
final_data = pd.concat([new_data,last_data],axis=0)

In [12]:
final_data.to_csv('../data/cumulative_data/{}.csv'.format(date_time), index=False)
final_data.to_csv('../data/cumulative_data/covid_last.csv'.format(date_time), index=False)


In [14]:

# last_data['Date_last_updated_AEDT'] = pd.DatetimeIndex(last_data['Date_last_updated_AEDT']) - pd.DateOffset(1)
# last_data['date'] = pd.DatetimeIndex(last_data['date']) - pd.DateOffset(1)

# last_data[cols].to_csv('../data/cumulative_data/covid_last.csv',index=False)

# Scrap data for US_CAN

In [32]:
# Test if we can scrap info from worldometers
# The communication with website is ok if the response is 200
US_Canada = "https://coronavirus.1point3acres.com/zh"
response2 = get(US_Canada, headers=headers)
response2

<Response [200]>

In [33]:
# Scrap all content from the website
html_soup2 = BeautifulSoup(response2.text, 'html.parser')

In [34]:
# Since they change class index everyday, this code is for finding the new index.
indexList = []
for span in html_soup2.find_all('span'):
    # Only retain 'span' that has contents
    if len(span.contents):
        # Since we only need to find index for table, use one of the table head as target word to locate index
        if span.contents[0] == 'Location':
            # Store the index inside a list
            indexList.append(span['class'][0])

In [35]:
# The first index is for US table and the 2nd index is for Canada table. Do not casr about the rest inside the list.
USindex, CANindex, _ = indexList

In [36]:
# Check if the index return right data
# html_soup2.find_all('span', class_=USindex)

In [37]:
# html_soup2.find_all('span', class_=CANindex)

In [38]:
len(html_soup2.find_all('span', class_=CANindex))

75

In [39]:
Locations = []
Confirmed = []
Recovered = []
Deaths = []
list1 = range(1, len(html_soup2.find_all('span', class_=USindex))-4, 5)
list2 = range(2, len(html_soup2.find_all('span', class_=USindex))-3, 5)
list3 = range(3, len(html_soup2.find_all('span', class_=USindex))-2, 5)
list4 = range(4, len(html_soup2.find_all('span', class_=USindex))-1, 5)

for index in list1:
    if len(html_soup2.find_all('span', class_=USindex)[index].contents):
        Locations.append(html_soup2.find_all('span', class_=USindex)[index].contents[0])
    else:
        Locations.append(0)
for index in list2:
    if len(html_soup2.find_all('span', class_=USindex)[index].contents):
        Confirmed.append(html_soup2.find_all('span', class_=USindex)[index].contents[0])
    else:
        Confirmed.append(0)
for index in list3:
    # They do not provide recovered cases number anymore.
    #if len(html_soup2.find_all('span', class_=USindex)[index].contents):
    #    Recovered.append(html_soup2.find_all('span', class_=USindex)[index].contents[0])
    #else:
    Recovered.append(0)
for index in list3:
    if len(html_soup2.find_all('span', class_=USindex)[index].contents):
        Deaths.append(html_soup2.find_all('span', class_=USindex)[index].contents[0])
    else:
        Deaths.append(0)
    
US_data = pd.DataFrame({'Province/State':Locations,
                        'Confirmed':Confirmed,
                        'Deaths':Deaths,
                        #'Recovered':Recovered,  
                            })

# Remove rows that are not data
US_data.drop(US_data[US_data['Deaths'] == 'Deaths'].index, axis=0, inplace=True)

# Replace Washington, D.C. as Washington DC
if 'Washington, D.C.' in list(US_data['Province/State']):
    US_data['Province/State'].replace({'Washington, D.C.':'Washington DC'}, inplace=True)

# Replace Washington as WA
if 'Washington' in list(US_data['Province/State']):
    US_data['Province/State'].replace({'Washington':'WA'}, inplace=True)
    
# Replace Grand Princess as From Grand Princess
if 'Grand Princess' in list(US_data['Province/State']):
    US_data['Province/State'].replace({'Grand Princess':'From Grand Princess'}, inplace=True)
    
# Replace Diamond Princess as From Diamond Princess cruise
if 'Diamond Princess' in list(US_data['Province/State']):
    US_data['Province/State'].replace({'Diamond Princess':'From Diamond Princess cruise'}, inplace=True)
    
# Assign 0 in column Province/State as unassigned
if 0 in list(US_data['Province/State']):
    US_data.at[US_data.loc[US_data['Province/State'] == 0,].index, 'Province/State'] = 'Unassigned'

In [40]:
# US_data

In [41]:
Locations = []
Confirmed = []
Recovered = []
Deaths = []
list1 = range(0, len(html_soup2.find_all('span', class_=CANindex))-4, 5)
list2 = range(1, len(html_soup2.find_all('span', class_=CANindex))-3, 5)
list3 = range(2, len(html_soup2.find_all('span', class_=CANindex))-2, 5)
list4 = range(3, len(html_soup2.find_all('span', class_=CANindex))-1, 5)

for index in list1:
    if len(html_soup2.find_all('span', class_=CANindex)[index].contents):
        Locations.append(html_soup2.find_all('span', class_=CANindex)[index].contents[0])
    else:
        Locations.append(0)
for index in list2:
    if len(html_soup2.find_all('span', class_=CANindex)[index].contents):
        Confirmed.append(html_soup2.find_all('span', class_=CANindex)[index].contents[0])
    else:
        Confirmed.append(0)
for index in list3:
    #. They do not provide recovered cases number
    #if len(html_soup2.find_all('span', class_=CANindex)[index].contents):
    #    Recovered.append(html_soup2.find_all('span', class_=CANindex)[index].contents[0])
    #else:
    Recovered.append(0)
for index in list3:
    if len(html_soup2.find_all('span', class_=CANindex)[index].contents):
        Deaths.append(html_soup2.find_all('span', class_=CANindex)[index].contents[0])
    else:
        Deaths.append(0)
    
CAN_data = pd.DataFrame({'Province/State':Locations,
                         'Confirmed':Confirmed,
                         'Deaths':Deaths,
                         #'Recovered':Recovered,  
                            })

# Remove rows that are not data
CAN_data.drop(CAN_data[CAN_data['Deaths'] == 'Deaths'].index, axis=0, inplace=True)

In [42]:
# CAN_data

In [43]:
US_Can_data = pd.concat([US_data, CAN_data], ignore_index=True)
US_Can_data = US_Can_data.apply(lambda x: x.str.strip())
# US_Can_data

In [44]:
nameList = pd.read_csv('../data/web_data/statesNameTranslation.csv')

In [45]:
US_Can_data_EN = pd.merge(US_Can_data, nameList, how = 'left', left_on = 'Province/State', right_on = 'English')
US_Can_data_EN = US_Can_data_EN.drop(['Chinese', 'Province/State', 'Abbr.'], axis=1)
US_Can_data_EN['Last Update'] = lastUpdateTime[1:]
US_Can_data_EN.rename(columns={'English':'Province/State'}, inplace=True)
US_Can_data_EN = US_Can_data_EN.drop(US_Can_data_EN[US_Can_data_EN['Province/State'] == 'Wuhan Evacuee'].index, axis=0)
columnOrder = ['Province/State', 'Country/Region', 'Last Update','Confirmed', 'Deaths', 'Recovered']

# US_Can_data_EN 

# China Data

In [ ]:
# Test if we can scrap info from worldometers
# The communication with website is ok if the response is 200
CHN = "https://ncov.dxy.cn/ncovh5/view/pneumonia?scene=2&clicktime=1579582238&enterid=1579582238&from=singlemessage&isappinstalled=0"
response3 = get(CHN, headers=headers)
response3.encoding='utf-8' ##去掉这句则乱码，加上则正常显示，其中utf-8是根据网页源代码中设置的编码格式来指定的  
response3

In [ ]:
# Scrap all content from the website
html_soup3 = BeautifulSoup(response3.text, 'html.parser')

In [ ]:
print(html_soup3.prettify())

In [ ]:
html_soup3.find_all('script', id='getAreaStat')[0].contents

In [ ]:
{"provinceName":"湖北省","provinceShortName":"湖北","currentConfirmedCount":19568,"confirmedCount":67707,"suspectedCount":0,"curedCount":45153,"deadCount":2986,"comment":"","locationId":420000,"statisticsData":"https://file1.dxycdn.com/2020/0223/618/3398299751673487511-135.json","cities":[{"cityName":"武汉","currentConfirmedCount":17634,"confirmedCount":49912,"suspectedCount":0,"curedCount":29908,"deadCount":2370,"locationId":420100},{"cityName":"孝感","currentConfirmedCount":369,"confirmedCount":3518,"suspectedCount":0,"curedCount":3024,"deadCount":125,"locationId":420900},{"cityName":"鄂州","currentConfirmedCount":352,"confirmedCount":1394,"suspectedCount":0,"curedCount":988,"deadCount":54,"locationId":420700},{"cityName":"随州","currentConfirmedCount":187,"confirmedCount":1307,"suspectedCount":0,"curedCount":1077,"deadCount":43,"locationId":421300},{"cityName":"宜昌","currentConfirmedCount":170,"confirmedCount":931,"suspectedCount":0,"curedCount":727,"deadCount":34,"locationId":420500},{"cityName":"荆州","currentConfirmedCount":155,"confirmedCount":1580,"suspectedCount":0,"curedCount":1376,"deadCount":49,"locationId":421000},{"cityName":"黄冈","currentConfirmedCount":151,"confirmedCount":2907,"suspectedCount":0,"curedCount":2631,"deadCount":125,"locationId":421100},{"cityName":"荆门","currentConfirmedCount":146,"confirmedCount":928,"suspectedCount":0,"curedCount":743,"deadCount":39,"locationId":420800},{"cityName":"黄石","currentConfirmedCount":95,"confirmedCount":1015,"suspectedCount":0,"curedCount":884,"deadCount":36,"locationId":420200},{"cityName":"十堰","currentConfirmedCount":93,"confirmedCount":672,"suspectedCount":0,"curedCount":571,"deadCount":8,"locationId":420300},{"cityName":"襄阳","currentConfirmedCount":82,"confirmedCount":1175,"suspectedCount":0,"curedCount":1055,"deadCount":38,"locationId":420600},{"cityName":"仙桃","currentConfirmedCount":53,"confirmedCount":575,"suspectedCount":0,"curedCount":501,"deadCount":21,"locationId":429004},{"cityName":"天门","currentConfirmedCount":24,"confirmedCount":496,"suspectedCount":0,"curedCount":457,"deadCount":15,"locationId":429006},{"cityName":"咸宁","currentConfirmedCount":21,"confirmedCount":836,"suspectedCount":0,"curedCount":801,"deadCount":14,"locationId":421200},{"cityName":"潜江","currentConfirmedCount":21,"confirmedCount":198,"suspectedCount":0,"curedCount":168,"deadCount":9,"locationId":429005},{"cityName":"恩施州","currentConfirmedCount":15,"confirmedCount":252,"suspectedCount":0,"curedCount":231,"deadCount":6,"locationId":422800},{"cityName":"神农架林区","currentConfirmedCount":0,"confirmedCount":11,"suspectedCount":0,"curedCount":11,"deadCount":0,"locationId":429021}]}